In [22]:
#import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pymongo
import datetime
from flask import Flask
import pymysql
from sqlalchemy import create_engine
import json

ModuleNotFoundError: No module named 'pymysql'

In [5]:
#import Redfin state data
raw_state_df = pd.read_csv("Resources/state_market_tracker.tsv000", sep='\t')
raw_state_df

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2019-10-01,2019-10-31,30,state,4,23,f,Oklahoma,NaN,Oklahoma,...,0.098560,0.209877,0.083561,0.053355,0.301282,-0.240768,-0.122115,South Region,NaN,2022-01-09 14:29:56
1,2021-07-01,2021-07-31,30,state,4,40,f,Vermont,NaN,Vermont,...,0.251473,0.133696,0.016402,-0.000131,0.424404,-0.064422,0.076156,Northeast Region,NaN,2022-01-09 14:29:56
2,2016-08-01,2016-08-31,30,state,4,10,f,New Hampshire,NaN,New Hampshire,...,0.089756,0.149826,0.009592,0.002687,0.206101,0.061385,0.016779,Northeast Region,NaN,2022-01-09 14:29:56
3,2013-04-01,2013-04-30,30,state,4,51,f,Mississippi,NaN,Mississippi,...,-0.008566,NaN,NaN,NaN,0.044777,-0.014869,-0.005006,South Region,NaN,2022-01-09 14:29:56
4,2019-12-01,2019-12-31,30,state,4,2,f,Missouri,NaN,Missouri,...,-0.017031,0.205339,-0.123381,-0.000410,0.252082,-0.116423,0.095007,Midwest Region,NaN,2022-01-09 14:29:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27073,2012-03-01,2012-03-31,30,state,4,6,f,Nebraska,NaN,Nebraska,...,0.021426,NaN,NaN,NaN,0.422780,0.042801,0.030384,Midwest Region,NaN,2022-01-09 14:29:56
27074,2016-07-01,2016-07-31,30,state,4,27,f,Idaho,NaN,Idaho,...,0.182390,NaN,NaN,NaN,0.534900,-0.117877,-0.004820,West Region,NaN,2022-01-09 14:29:56
27075,2020-11-01,2020-11-30,30,state,4,42,f,Virginia,NaN,Virginia,...,0.218939,0.126582,-0.048418,-0.006751,0.380567,-0.152381,0.194669,South Region,NaN,2022-01-09 14:29:56
27076,2021-08-01,2021-08-31,30,state,4,40,f,Vermont,NaN,Vermont,...,0.234272,0.158853,0.018284,0.022594,0.381659,-0.018233,0.098021,Northeast Region,NaN,2022-01-09 14:29:56


In [15]:
#drop unnecessary columns
state_df = raw_state_df[["period_end", "state_code", "median_sale_price", "median_ppsf"]]
state_df

,period_end,state_code,median_sale_price,median_ppsf
0,2019-10-31,OK,162200,77.0
1,2021-07-31,VT,317900,177.0
2,2016-08-31,NH,200100,155.0
3,2013-04-30,MS,129500,69.0
4,2019-12-31,MO,152000,135.0
...,...,...,...,...
27073,2012-03-31,NE,133000,74.0
27074,2016-07-31,ID,178400,135.0
27075,2020-11-30,VA,264200,122.0
27076,2021-08-31,VT,326800,180.0


In [10]:
#view column types
state_df.dtypes

period_end            object
state                 object
median_sale_price      int64
median_ppsf          float64
dtype: object

In [16]:
#cast columns to strings and date/time
state_df['period_end'] = pd.to_datetime(state_df['period_end'], format='%Y-%m-%d')
state_df['state_code'] = state_df['state_code'].astype('string')
state_df.dtypes

/var/folders/hd/ltllzg1x3cd1p6_w0hms1sg00000gn/T/ipykernel_30866/513362087.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df['period_end'] = pd.to_datetime(state_df['period_end'], format='%Y-%m-%d')
/var/folders/hd/ltllzg1x3cd1p6_w0hms1sg00000gn/T/ipykernel_30866/513362087.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df['state_code'] = state_df['state_code'].astype('string')


period_end           datetime64[ns]
state_code                   string
median_sale_price             int64
median_ppsf                 float64
dtype: object

In [34]:
#drop years 2018 and older
state_df = state_df[state_df.period_end.dt.year > 2018]
state_df

,period_end,state_code,median_sale_price,median_ppsf
0,2019-10-31,OK,162200,77.0
1,2021-07-31,VT,317900,177.0
4,2019-12-31,MO,152000,135.0
5,2019-07-31,NM,385500,218.0
9,2020-03-31,OH,156100,111.0
...,...,...,...,...
27068,2020-04-30,NE,206700,103.0
27070,2019-12-31,OK,169500,109.0
27071,2019-01-31,AZ,262600,151.0
27075,2020-11-30,VA,264200,122.0


In [39]:
#grab mean sale price by state
state_df_mean = state_df.groupby(['state_code']).mean()
state_df_mean.sort_values(by='median_sale_price').head()

,median_sale_price,median_ppsf
state_code,,
OK,162056.111111,96.811111
OH,169773.888889,103.511111
MS,173332.022472,104.073034
IN,179582.876712,101.000000
MO,181113.333333,114.316667


In [40]:
#grab median sale price by state
state_df_median = state_df.groupby(['state_code']).median()
state_df_median.sort_values(by='median_sale_price').head()

,median_sale_price,median_ppsf
state_code,,
OK,171750.0,97.5
OH,173250.0,110.0
MO,179300.0,119.0
MS,179500.0,105.0
KY,183650.0,114.0


In [ ]:
state_df_median.sort_values(by='median_ppsf').head()

In [14]:
#import data scientist salary information
raw_salary_df = pd.read_csv("Resources/raw_salary_data_scientist.csv")
raw_salary_df

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,tensor,hadoop,tableau,bi,flink,mongo,google_an,job_title_sim,seniority_by_title,Degree
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 - 1000,1973,...,0,0,1,1,0,0,0,data scientist,na,M
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+,1984,...,0,0,0,0,0,0,0,data scientist,na,M
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 - 1000,2010,...,0,0,0,0,0,0,0,data scientist,na,M
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 - 5000,1965,...,0,0,0,0,0,0,0,data scientist,na,na
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 - 200,1998,...,0,0,0,0,0,0,0,data scientist,na,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,950,"Sr Scientist, Immuno-Oncology - Oncology",$58K-$111K (Glassdoor est.),Site Name: USA - Massachusetts - Cambridge\nPo...,3.9,GSK\n3.9,"Cambridge, MA","Brentford, United Kingdom",10000+,1830,...,0,0,0,0,0,0,0,other scientist,sr,M
738,951,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrite\n4.4,"Nashville, TN","San Francisco, CA",1001 - 5000,2006,...,0,1,0,0,0,0,0,data engineer,sr,na
739,952,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute\n2.6,"Pittsburgh, PA","Pittsburgh, PA",501 - 1000,1984,...,0,0,0,0,0,0,0,other scientist,na,P
740,953,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LLC\n3.2","Allentown, PA","Chadds Ford, PA",1 - 50,-1,...,0,0,0,0,0,0,0,data scientist,na,na


In [19]:
#drop unneccesary columns
salary_df = raw_salary_df[['Rating', 'Lower Salary', 'Upper Salary', 'Avg Salary(K)', 'Job Location', 'job_title_sim', 'seniority_by_title']]
salary_df

,Rating,Lower Salary,Upper Salary,Avg Salary(K),Job Location,job_title_sim,seniority_by_title
0,3.8,53,91,72.0,NM,data scientist,na
1,3.4,63,112,87.5,MD,data scientist,na
2,4.8,80,90,85.0,FL,data scientist,na
3,3.8,56,97,76.5,WA,data scientist,na
4,2.9,86,143,114.5,NY,data scientist,na
...,...,...,...,...,...,...,...
737,3.9,58,111,84.5,MA,other scientist,sr
738,4.4,72,133,102.5,TN,data engineer,sr
739,2.6,56,91,73.5,PA,other scientist,na
740,3.2,95,160,127.5,PA,data scientist,na


In [20]:
#get column types
salary_df.dtypes

Rating                float64
Lower Salary            int64
Upper Salary            int64
Avg Salary(K)         float64
Job Location           object
job_title_sim          object
seniority_by_title     object
dtype: object

In [21]:
#convery types to strings
salary_df['Job Location'] = salary_df['Job Location'].astype('string')
salary_df['job_title_sim'] = salary_df['job_title_sim'].astype('string')
salary_df['seniority_by_title'] = salary_df['seniority_by_title'].astype('string')
salary_df.dtypes

/var/folders/hd/ltllzg1x3cd1p6_w0hms1sg00000gn/T/ipykernel_30866/2571449120.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salary_df['Job Location'] = salary_df['Job Location'].astype('string')
/var/folders/hd/ltllzg1x3cd1p6_w0hms1sg00000gn/T/ipykernel_30866/2571449120.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salary_df['job_title_sim'] = salary_df['job_title_sim'].astype('string')
/var/folders/hd/ltllzg1x3cd1p6_w0hms1sg00000gn/T/ipykernel_30866/2571449120.py:4: SettingWithCopyWarning: 
A val

Rating                float64
Lower Salary            int64
Upper Salary            int64
Avg Salary(K)         float64
Job Location           string
job_title_sim          string
seniority_by_title     string
dtype: object

In [28]:
#determine values in seniority by title
salary_df['seniority_by_title'].value_counts()

na    519
sr    220
jr      3
Name: seniority_by_title, dtype: Int64

In [31]:
#drop senior and junior positions
salary_df = salary_df[salary_df.seniority_by_title != "sr"]
salary_df = salary_df[salary_df.seniority_by_title != "jr"]
salary_df['seniority_by_title'].value_counts()

na    519
Name: seniority_by_title, dtype: Int64

In [23]:
#re-name columns job location to state_code to match other data frame
salary_df = salary_df.rename(columns = {'Job Location': 'state_code'})
salary_df

,Rating,Lower Salary,Upper Salary,Avg Salary(K),state_code,job_title_sim,seniority_by_title
0,3.8,53,91,72.0,NM,data scientist,na
1,3.4,63,112,87.5,MD,data scientist,na
2,4.8,80,90,85.0,FL,data scientist,na
3,3.8,56,97,76.5,WA,data scientist,na
4,2.9,86,143,114.5,NY,data scientist,na
...,...,...,...,...,...,...,...
737,3.9,58,111,84.5,MA,other scientist,sr
738,4.4,72,133,102.5,TN,data engineer,sr
739,2.6,56,91,73.5,PA,other scientist,na
740,3.2,95,160,127.5,PA,data scientist,na


In [24]:
merged_df = pd.merge(state_df, salary_df, how='inner', on='state_code')

In [25]:
merged_df

,period_end,state_code,median_sale_price,median_ppsf,Rating,Lower Salary,Upper Salary,Avg Salary(K),job_title_sim,seniority_by_title
0,2019-12-31,MO,152000,135.0,4.7,111,176,143.5,data scientist,na
1,2019-12-31,MO,152000,135.0,2.7,50,102,76.0,other scientist,na
2,2019-12-31,MO,152000,135.0,3.8,26,55,40.5,Data scientist project manager,na
3,2019-12-31,MO,152000,135.0,4.1,44,78,61.0,analyst,na
4,2019-12-31,MO,152000,135.0,4.1,44,78,61.0,analyst,na
...,...,...,...,...,...,...,...,...,...,...
441482,2019-01-31,AZ,262600,151.0,3.5,57,80,68.5,data engineer,na
441483,2019-01-31,AZ,262600,151.0,3.4,90,110,100.0,data engineer,sr
441484,2019-01-31,AZ,262600,151.0,3.4,42,79,60.5,data engineer,na
441485,2019-01-31,AZ,262600,151.0,3.5,57,80,68.5,data engineer,na
